In [1]:
#code for displaying responses
from textblob import TextBlob
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intent(1).json').read())
words = pickle.load(open('words_erp.pkl','rb'))
classes = pickle.load(open('classes_erp.pkl','rb'))
model = load_model('chatbot_erp.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i,word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    sentence=str(sentence)
    print(sentence)
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD ]
#     print(results)
    
    results.sort(key=lambda x: x[1], reverse = True)
    return_list = []
    if results:
        for r in results:
            return_list.append({'intent':classes[r[0]],'probability':str([r[1]])})
        print(return_list)
        return return_list[0]  #i am only printing response of higher probability tag
    else:
        return return_list

def get_response(intents,tag):
    res=""
    print(tag)
    for i in intents['intents']:
        if i['tag']==tag:
            print("_______hi____")
            return(random.choice(i['responses']))
        else:
            continue          
            
    
if __name__ == "__main__":
    while True:
        count=1
        txt = input("")
        message=TextBlob(txt).correct()
        prediction = predict_class(message)
        if prediction:
            tag= prediction['intent']
            res = get_response(intents,tag)
            if res:
                print(res)
        else:
            print("Sorry, I don't have any information about that.") 


KeyboardInterrupt



In [19]:
from tkinter import scrolledtext, Entry, Button, END
from textblob import TextBlob
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import datetime

# Load intents, words, and classes from pickle files
intents = json.loads(open('intent(1).json').read())
words = pickle.load(open('words_erp.pkl', 'rb'))
classes = pickle.load(open('classes_erp.pkl', 'rb'))
model = load_model('chatbot_erp.h5')
domain_specific_words = ["proctor","parent", "'s", 'Au', 'I', 'ID', 'USN', 'a', 'address', 'admission', 'app', 'appreciate', 'are', 'assessment', 'attendance', 'attendnce', 'auid', 'birth', 'book', 'built', 'bye', 'can', 'catch', 'change', 'check', 'class', 'classroom', 'collage', 'college', 'contact', 'could', 'course', 'coursera', 'created', 'daily', 'dance', 'date', 'day', 'department', 'detail', 'determine', 'did', 'displayed', 'do', 'email', 'emmergency', 'erp', 'exam', 'farewell', 'fee', 'feedback', 'find', 'for', 'from', 'give', 'going', 'good', 'goodbye', 'hand', 'have', 'hello', 'help', 'helpful', 'hey', 'hi', 'history', 'how', 'i', 'ia', 'icecream', 'id', 'idiot', 'in', 'information', 'internal', 'internals', 'is', 'it', 'know', 'later', 'location', 'logout', 'love', 'made', 'mark', 'me', 'modify', 'much', 'my', 'name', 'need', 'nonsense', 'notification', 'now', 'number', 'of', 'parent', 'password', 'pay', 'payment', 'phone', 'please', 'proctor', 'profile', 'rate', 'receipt', 'record', 'reset', 'revoir', 'room', 'score', 'section', 'see', 'semester', 'should', 'sing', 'song', 'stepts', 'student', 'support', 'table', 'teacher', 'term', 'thank', 'thanks', 'that', 'the', 'there', 'time', 'timetable', 'to', 'transaction', 'udpate', 'uniform', 'up', 'update', 'usn', 'very', 'view', 'want', 'were', 'what', 'where', 'which', 'who', 'wise', 'you', 'your']

# Initialize WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to clean up the sentence
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

# Function to create bag of words
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

# Function to predict intent class
def predict_class(sentence):
    sentence = str(sentence)
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.5  # Increased from 0.6 to 0.7
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    if results:
        return_list.append({'intent': classes[results[0][0]], 'probability': str([results[0][1]])})
        print(return_list)
        return return_list[0]
    else:
        return_list.append({'intent': 'unrecognized', 'probability': '0.0'})
        return return_list[0]

# Function to get response based on intent
def get_response(intents, tag):
    for i in intents['intents']:
        if i['tag'] == tag:
            print("_______hi____")
            return random.choice(i['responses'])
    print("_______hi____")
    return "Sorry, I don't have any information about that."

# Function to handle unrecognized intent
def handle_unrecognized_intent():
    print("Sorry, I don't have any information about that.")

# Function to send a message and get a response
def send_message():
    user_input = entry.get()
    
    # Split the input into words
    words_in_input = user_input.split()

    # Apply TextBlob correction selectively
    corrected_words = []
    for word in words_in_input:
        if word.lower() not in domain_specific_words:
            corrected_word = str(TextBlob(word).correct())
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Join the corrected words back into a sentence
    corrected_input = ' '.join(corrected_words)

    prediction = predict_class(corrected_input)
    tag = prediction['intent']

    # Initialize response variable
    response = ""
    if tag == 'unrecognized':
        response = "Sorry, I don't have any information about that."
    else:
        # Get the response based on the intent
        response = get_response(intents, tag)

    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    chat_display.insert(tk.END, f"{current_time} - You: " + user_input + "\n")
    chat_display.insert(tk.END, f"{current_time} - Chatbot: " + response +"\n"+"------------------------------------------------"+ "\n")

    # Clear the entry field after sending the message
    entry.delete(0, tk.END)

# Create the main window
window = tk.Tk()
window.title("Chatbot Interface")

# Apply QSS styles from styles.qss file using configure method
window.configure(background='#2E2E2E')

# Create and configure the chat display
chat_display = scrolledtext.ScrolledText(window, width=50, height=20, wrap=tk.WORD)
chat_display.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Create the entry for user input
entry = Entry(window, width=40)
entry.grid(row=1, column=0, padx=10, pady=10)
entry.configure(background='#333333', fg='#FFFFFF', bd=2, relief='solid', insertbackground='#4CAF50')

# Create the send button
send_button = Button(window, text="Send", command=send_message)
send_button.grid(row=1, column=1, padx=10, pady=10)
send_button.configure(background='#4CAF50', fg='white', bd=0, padx=20, pady=10, font=('Arial', 16), cursor='hand2')

# Start the GUI main loop
window.mainloop()


1/1 [==============================] - 0s 112ms/step
[{'intent': 'greeting', 'probability': '[0.9999987]'}]
_______hi____
1/1 [==============================] - 0s 58ms/step
[{'intent': 'attendance_check', 'probability': '[0.99925536]'}]
_______hi____
1/1 [==============================] - 0s 25ms/step
[{'intent': '18', 'probability': '[0.99999666]'}]
_______hi____
1/1 [==============================] - 0s 59ms/step
[{'intent': 'attendance_check', 'probability': '[0.9832897]'}]
_______hi____
1/1 [==============================] - 0s 62ms/step
[{'intent': '7', 'probability': '[0.8550381]'}]
_______hi____


In [7]:
pip install --upgrade pandas


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 10.8/10.8 MB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 346.6/346.6 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
